Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Create a function that defines the model. Add L2 regularisation to the loss function by adding the L2 norm of the Weights and the Biases to the original loss function. Scale the norms by a parameter beta indicating the amount of regularisation.

In [5]:
def logistic_regression(beta=0.1, batch_size=128):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size),
                                         name='train_dataset')
        tf_train_labels = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels),
                                        name='train_labels')
        tf_valid_dataset = tf.constant(valid_dataset, dtype=tf.float32)
        tf_test_dataset = tf.constant(test_dataset, dtype=tf.float32)

        #Variables
        weights = tf.Variable(tf.truncated_normal([image_size**2, num_labels]))
        biases = tf.Variable(tf.zeros([num_labels]))

        #Training Computation
        logits = tf.matmul(tf_train_dataset, weights) + biases
        loss = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        loss = loss + beta*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
        loss = tf.reduce_mean(loss, name='loss')
        
        #Optimiser
        optimiser = tf.train.GradientDescentOptimizer(0.5).minimize(loss, name='optimiser')
        
        #Predictions
        train_prediction = tf.nn.softmax(logits, name='train_prediction')
        valid_prediction = tf.nn.softmax(tf.matmul(valid_dataset, weights) + biases, name='valid_prediction')
        test_prediction = tf.nn.softmax(tf.matmul(test_dataset, weights) + biases, name='test_prediction')
    return graph

Same as logisitc regression. Create a function definining the model and penalise the Weights and the Biases.

In [26]:
def neural_network(beta=0.1, batch_size=128):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size),
                                         name='train_dataset')
        tf_train_labels = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels),
                                         name='train_labels')
        tf_valid_dataset = tf.constant(valid_dataset, dtype=tf.float32)
        tf_test_dataset = tf.constant(test_dataset, dtype=tf.float32)

        #Variables
        weights_1 = tf.Variable(tf.truncated_normal([image_size**2, 1024]))
        biases_1 = tf.Variable(tf.zeros([1024]))
        weights_2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))

        #Training Computation
        layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
        activation_1 = tf.nn.relu(layer_1)
        logits = tf.matmul(activation_1, weights_2) + biases_2
        loss = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        loss = loss + beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1)
                            + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
        loss = tf.reduce_mean(loss, name='loss')

        #Optimiser
        optimiser = tf.train.GradientDescentOptimizer(0.5).minimize(loss, name='optimiser')

        #Predictions
        train_prediction = tf.nn.softmax(logits, name='train_prediction')
        valid_activation = tf.nn.relu(tf.matmul(valid_dataset, weights_1) + biases_1)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_activation, weights_2) + biases_2,
                                         name='valid_prediction')
        test_activation = tf.nn.relu(tf.matmul(test_dataset, weights_1) + biases_1)
        test_prediction = tf.nn.softmax(tf.matmul(test_activation, weights_2) + biases_2,
                                       name='test_prediction')
    return graph

Below is just a function that runs tensorflow graphs (models).

In [7]:
def run_graph(graph, num_batches=1000, num_steps=3000, batch_size=128):
    num_steps += 1
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        tf_train_dataset=graph.get_tensor_by_name("train_dataset:0")
        tf_train_labels=graph.get_tensor_by_name("train_labels:0")
        optimiser=graph.get_operation_by_name("optimiser")
        loss=graph.get_tensor_by_name("loss:0")
        train_prediction=graph.get_tensor_by_name("train_prediction:0")
        valid_prediction=graph.get_tensor_by_name("valid_prediction:0")
        test_prediction=graph.get_tensor_by_name("test_prediction:0")
        for step in xrange(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            if offset % num_batches*batch_size == 0:
                offset = 0
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            #Feed Dictionary
            feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
            _, l, predictions = session.run([optimiser, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [33]:
regression = logistic_regression()
run_graph(regression)

Initialized
Minibatch loss at step 0: 315.435669
Minibatch accuracy: 11.7%
Validation accuracy: 14.8%
Minibatch loss at step 500: 1.094512
Minibatch accuracy: 75.8%
Validation accuracy: 68.9%
Minibatch loss at step 1000: 1.058483
Minibatch accuracy: 76.6%
Validation accuracy: 75.5%
Minibatch loss at step 1500: 1.009662
Minibatch accuracy: 82.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.962315
Minibatch accuracy: 76.6%
Validation accuracy: 75.5%
Minibatch loss at step 2500: 1.096092
Minibatch accuracy: 69.5%
Validation accuracy: 74.9%
Minibatch loss at step 3000: 1.017998
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Test accuracy: 83.8%


In [34]:
network = neural_network()
run_graph(network)

Initialized
Minibatch loss at step 0: 31826.207031
Minibatch accuracy: 3.9%
Validation accuracy: 28.8%
Minibatch loss at step 500: 1.422327
Minibatch accuracy: 78.9%
Validation accuracy: 74.2%
Minibatch loss at step 1000: 1.387566
Minibatch accuracy: 79.7%
Validation accuracy: 76.5%
Minibatch loss at step 1500: 1.386933
Minibatch accuracy: 81.2%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 1.300252
Minibatch accuracy: 85.2%
Validation accuracy: 78.9%
Minibatch loss at step 2500: 1.408599
Minibatch accuracy: 75.8%
Validation accuracy: 73.5%
Minibatch loss at step 3000: 1.360341
Minibatch accuracy: 83.6%
Validation accuracy: 78.1%
Test accuracy: 83.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Remove regularisation by setting `beta=0`. Train on one batch only. Accuracy on training data is perfect. Accuracy on validation and test data is worse.

In [71]:
regression = logistic_regression(beta=0)
run_graph(regression, 1)

Initialized
Minibatch loss at step 0: 21.375443
Minibatch accuracy: 3.1%
Validation accuracy: 10.6%
Minibatch loss at step 1000: 0.001544
Minibatch accuracy: 100.0%
Validation accuracy: 57.4%
Minibatch loss at step 2000: 0.000799
Minibatch accuracy: 100.0%
Validation accuracy: 58.3%
Minibatch loss at step 3000: 0.000546
Minibatch accuracy: 100.0%
Validation accuracy: 58.8%
Test accuracy: 65.2%


Remove regularisation by setting `beta=0`. Train on one batch only. Accuracy on training data is perfect. Accuracy on validation and test data is worse.

In [73]:
network = neural_network(beta=0)
run_graph(network, 1)

Initialized
Minibatch loss at step 0: 393.473694
Minibatch accuracy: 6.2%
Validation accuracy: 34.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 65.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 65.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 65.9%
Test accuracy: 71.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Add dropout on the activations of the hidden layer during training. Accuracy on validation and testing data for extreme case improves.

In [25]:
def neural_network_with_dropout(beta=0.1, batch_size=128):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size),
                                         name='train_dataset')
        tf_train_labels = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels),
                                         name='train_labels')
        tf_valid_dataset = tf.constant(valid_dataset, dtype=tf.float32)
        tf_test_dataset = tf.constant(test_dataset, dtype=tf.float32)

        #Variables
        weights_1 = tf.Variable(tf.truncated_normal([image_size**2, 1024]))
        biases_1 = tf.Variable(tf.zeros([1024]))
        weights_2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
        biases_2 = tf.Variable(tf.zeros([num_labels]))

        #Training Computation
        layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
        activation_1 = tf.nn.relu(layer_1)
        activation_1 = tf.nn.dropout(activation_1, 0.5)
        logits = tf.matmul(activation_1, weights_2) + biases_2
        loss = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        loss = loss + beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1)
                            + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
        loss = tf.reduce_mean(loss, name='loss')

        #Optimiser
        optimiser = tf.train.GradientDescentOptimizer(0.5).minimize(loss, name='optimiser')

        #Predictions
        train_prediction = tf.nn.softmax(logits, name='train_prediction')
        valid_activation = tf.nn.relu(tf.matmul(valid_dataset, weights_1) + biases_1)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_activation, weights_2) + biases_2,
                                         name='valid_prediction')
        test_activation = tf.nn.relu(tf.matmul(test_dataset, weights_1) + biases_1)
        test_prediction = tf.nn.softmax(tf.matmul(test_activation, weights_2) + biases_2,
                                       name='test_prediction')
    return graph

In [20]:
network_with_dropout = neural_network_with_dropout(beta=0)
run_graph(network_with_dropout, 1)

Initialized
Minibatch loss at step 0: 460.577209
Minibatch accuracy: 10.2%
Validation accuracy: 37.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Test accuracy: 77.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Add an extra layer with droupout and utilise learning rate decay. Accuracy on testing data improves. It is likely that accuracy will improve more if dropout is not used because there is a large number of training samples.

In [8]:
def deep_neural_network_with_dropout(beta=0.1, do_prob=0.6, batch_size=128):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(dtype=tf.float32, shape=(batch_size, image_size*image_size),
                                         name='train_dataset')
        tf_train_labels = tf.placeholder(dtype=tf.float32, shape=(batch_size, num_labels),
                                         name='train_labels')
        tf_valid_dataset = tf.constant(valid_dataset, dtype=tf.float32)
        tf_test_dataset = tf.constant(test_dataset, dtype=tf.float32)

        #Variables
        global_step = tf.Variable(0)
        
        weights_1 = tf.Variable(tf.truncated_normal([image_size**2, 1024]))
        biases_1 = tf.Variable(tf.zeros([1024]))
        weights_2 = tf.Variable(tf.truncated_normal([1024, 1024]))
        biases_2 = tf.Variable(tf.zeros([1024]))
        weights_3 = tf.Variable(tf.truncated_normal([1024, num_labels]))
        biases_3 = tf.Variable(tf.zeros([num_labels]))

        
        #Training Computation
        layer_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
        activation_1 = tf.nn.relu(layer_1)
        activation_1 = tf.nn.dropout(activation_1, do_prob)
        layer_2 = tf.matmul(activation_1, weights_2) + biases_2
        activation_2 = tf.nn.relu(layer_2)
        activation_2 = tf.nn.dropout(activation_2, do_prob)
        logits = tf.matmul(activation_2, weights_3) + biases_3
        loss = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
        loss = loss + beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1)
                            + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2)
                            + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3))
        loss = tf.reduce_mean(loss, name='loss')

        #Learning Rate
        learning_rate = tf.train.exponential_decay(0.01, global_step=global_step,
                                                   decay_steps=1000, decay_rate=0.9, staircase=True)
        
        #Optimiser
        optimiser = tf.train.GradientDescentOptimizer(learning_rate)
        optimiser.minimize(loss, global_step=global_step, name='optimiser')

        #Predictions
        train_prediction = tf.nn.softmax(logits, name='train_prediction')
        valid_activation_1 = tf.nn.relu(tf.matmul(valid_dataset, weights_1) + biases_1)
        valid_activation_2 = tf.nn.relu(tf.matmul(valid_activation_1, weights_2) + biases_2)
        valid_prediction = tf.nn.softmax(tf.matmul(valid_activation_2, weights_3) + biases_3,
                                         name='valid_prediction')
        test_activation_1 = tf.nn.relu(tf.matmul(test_dataset, weights_1) + biases_1)
        test_activation_2 = tf.nn.relu(tf.matmul(test_activation_1, weights_2) + biases_2)
        test_prediction = tf.nn.softmax(tf.matmul(test_activation_2, weights_3) + biases_3,
                                       name='test_prediction')
    return graph

In [ ]:
deep_network = deep_neural_network_with_dropout(0.1, 0.6)
run_graph(deep_network, num_steps=10000)

Initialized
Minibatch loss at step 0: 83733.164062
Minibatch accuracy: 15.6%
Validation accuracy: 17.3%
Minibatch loss at step 500: 26281.787109
Minibatch accuracy: 73.4%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 9634.846680
Minibatch accuracy: 76.6%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 3911.721436
Minibatch accuracy: 77.3%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 1589.730469
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 707.795776
Minibatch accuracy: 75.8%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 315.645844
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Minibatch loss at step 3500: 153.094193
Minibatch accuracy: 78.1%
Validation accuracy: 80.6%
Minibatch loss at step 4000: 74.709023
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 4500: 39.620338
Minibatch accuracy: 78.1%
Validation accuracy: 80.2%
Minibatch loss at step 5000: 21.321371
Minibatch accuracy

In [ ]:
deep_network = deep_neural_network_with_dropout(0.1, 1)
run_graph(deep_network, num_steps=10000)